In [45]:
from langchain_openai import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_core.output_parsers import StrOutputParser
from langchain_community.retrievers import WikipediaRetriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import RunnablePassthrough
import os

In [7]:
# SECURE THIS KEY!
os.environ["AZURE_OPENAI_API_KEY"] = ""
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://azqoreai.openai.azure.com/"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "gpt-35-turbo-16k_azqore_ai"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-02-01"

In [41]:
model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)
output_parser= StrOutputParser()

#this retriever (more on this later) let us access Wikipedia
retriever = WikipediaRetriever()

In [34]:

prompt = PromptTemplate(
    template="Create a summary in 10 lines about this topic {input} using only the data provided here : {context}"
)


#this chain will invoke the llm passing the documents retrieved from from wikipedia as input in the "context" variable
llm_chain = create_stuff_documents_chain(model, prompt)

#this chain will invoke the wikipedia retriever then pass the result to llm_chain
chain = create_retrieval_chain(retriever, llm_chain)


output = chain.invoke({
    "input": "euro 2024",
})
output_parser.invoke(output['answer'])



"UEFA Euro 2024 is the 17th edition of the European Championship for men's national teams, organized by UEFA. It is being hosted by Germany from 14 June to 14 July 2024. The tournament features 24 teams, with Georgia making their debut in the finals. This is the third time matches are played on German territory and the first time in what was formerly East Germany. Germany is serving as the sole host nation for the first time since the 2006 FIFA World Cup. The tournament returned to its four-year cycle after being postponed in 2020 due to the COVID-19 pandemic. Italy, the defending champions, were eliminated in the round of 16 by Switzerland. The tournament is being held in various stadiums across Germany, with Berlin hosting the final match. The qualifying tournament for Euro 2024 took place from March 2023 to March 2024, with the top two teams from each group and the four best third-placed teams joining Germany in the final tournament."

In [40]:

prompt = PromptTemplate(
    template="{input}"
)

#this chain will invoke the llm passing the documents retrieved from from wikipedia as input in the "context" variable
chain = create_retrieval_chain(retriever, prompt)

output = chain.invoke({
    "input": "euro 2024",
})
output["context"]

[Document(metadata={'title': 'UEFA Euro 2024', 'summary': "The 2024 UEFA European Football Championship, commonly referred to as UEFA Euro 2024 (stylised as UEFA EURO 2024), or simply Euro 2024, is the ongoing 17th edition of the UEFA European Championship, the quadrennial international football  championship organised by UEFA for the European men's national teams of its member associations. Germany is hosting the tournament, which is taking place from 14 June to 14 July 2024. The tournament comprises 24 teams, with Georgia being the only team making their European Championship finals debut. \nIt is the third time that European Championship matches are played on German territory and the second time in reunified Germany, as West Germany hosted the tournament's 1988 edition, and four matches of the multi-national Euro 2020 were played in Munich. It is the first time the competition is held in what was formerly East Germany, with Leipzig as a host city, as well as the first major tourname

In [69]:
from langchain_core.prompts import ChatPromptTemplate

template="Create a summary in 10 lines about this topic {input} using only the data provided here : {context}"

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever, "input": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

chain.invoke("euro 2024")


'The UEFA Euro 2024 is the ongoing 17th edition of the UEFA European Championship. It is being hosted by Germany and is taking place from 14 June to 14 July 2024. The tournament features 24 teams, with Georgia making their debut in the European Championship finals. This is the third time that European Championship matches are being played on German territory and the first time in what was formerly East Germany. The tournament returned to its usual four-year cycle after the 2020 edition was postponed due to the COVID-19 pandemic. Italy, the defending champions, were eliminated in the round of 16 by Switzerland. The knockout stage of the tournament began on 29 June 2024 and will end with the final on 14 July 2024 at Olympiastadion in Berlin. The qualifying tournament for Euro 2024 involved 53 UEFA member associations and was played from March 2023 to March 2024. The group stage determined 20 of the 23 teams that would join host Germany in the final tournament, while the remaining three t